# Выбор локации для скважины
___
# Choice of location for the well

Основная задача - принять решение о локализации потенциальной скважины.

Для ее решения имеются данные о пробы нефти в 3-х регионах. В каждом из регионом по 10000 месторождений, в которых были произведены замеры качества и объема.

Необходимо построить модель машинного обучения, с помощью которой впоследствии будет возможно определенеи региона, имеющего наибольший экономический потенциал.
___
The main task is to decide on the localization of a potential well.

To solve it, there are data on oil samples in 3 regions. There are 10,000 deposits in each region, in which measurements of quality and volume were made.

It is necessary to build a machine learning model, with the help of which it will subsequently be possible to determine the region with the greatest economic potential.

## Загрузка и подготовка данных
___
## Loading and preparing data

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import re
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from random import sample
from numpy.random import RandomState

In [2]:
df_1 = pd.read_csv('geo_data_0.csv')

df_2 = pd.read_csv('geo_data_1.csv')

df_3 = pd.read_csv('geo_data_2.csv')

In [3]:
df_1.head(5)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
df_1.duplicated().mean()

0.0

In [6]:
df_1.isna().mean()*100

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

In [7]:
df_1 = df_1.drop(['id'], axis = 1)

In [8]:
df_2.head(5)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [9]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
df_2.duplicated().mean()

0.0

In [11]:
df_2.isna().mean()*100

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

In [12]:
df_2 = df_2.drop(['id'], axis = 1)

In [13]:
df_3.head(5)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [14]:
df_3.duplicated().mean()

0.0

In [15]:
df_3.isna().mean()*100

id         0.0
f0         0.0
f1         0.0
f2         0.0
product    0.0
dtype: float64

In [16]:
df_3 = df_3.drop(['id'], axis = 1)

Данные по регионам чистые, датасеты не обладают дубликатами, пропусками. Столбцы "id" были удалены, так как являются уникальными для каждого клиента и будут мешать построению функционирующей модели.
___
The data by region is clean, the datasets do not have duplicates or gaps. The "id" columns have been removed as they are unique to each client and will interfere with building a functioning model.

## Обучение и проверка модели
___
## Train and validate the model

Выделим признаки и целевые показатели.
___
Let's highlight the signs and targets.

In [17]:
features_1 = df_1.drop('product', axis=1)
target_1 = df_1['product']

In [18]:
features_2 = df_2.drop('product', axis=1)
target_2 = df_2['product']

In [19]:
features_3 = df_3.drop('product', axis=1)
target_3 = df_3['product']

Разделим выборки.
___
Let's split the samples.

In [20]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, test_size=0.25, random_state=12345)

In [21]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, test_size=0.25, random_state=12345)

In [22]:
features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(features_3, target_3, test_size=0.25, random_state=12345)

Масштабируем данные для улучшения результата.
___
We scale the data to improve the result.

In [23]:
numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train_1[numeric])
features_train_1[numeric] = scaler.transform(features_train_1[numeric])
features_valid_1[numeric] = scaler.transform(features_valid_1[numeric])   

C:\Users\kiril\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\kiril\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [24]:
numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train_2[numeric])
features_train_2[numeric] = scaler.transform(features_train_2[numeric])
features_valid_2[numeric] = scaler.transform(features_valid_2[numeric])   

C:\Users\kiril\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\kiril\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [25]:
numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train_3[numeric])
features_train_3[numeric] = scaler.transform(features_train_3[numeric])
features_valid_3[numeric] = scaler.transform(features_valid_3[numeric])   

C:\Users\kiril\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\kiril\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Создадим функцию для упрощения задачи.
___
Let's create a function to simplify the task.

In [26]:
def lin_reg(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    return predictions_valid

In [27]:
predictions_valid_1 = lin_reg(features_train_1, target_train_1, features_valid_1)

In [28]:
predictions_valid_2 = lin_reg(features_train_2, target_train_2, features_valid_2)

In [29]:
predictions_valid_3 = lin_reg(features_train_3, target_train_3, features_valid_3)

In [30]:
predictions_valid_1.mean()

92.59256778438035

In [31]:
predictions_valid_2.mean()

68.728546895446

In [32]:
predictions_valid_3.mean()

94.96504596800489

In [33]:
target_valid_1.mean()

92.07859674082941

In [34]:
target_valid_2.mean()

68.72313602437494

In [35]:
target_valid_3.mean()

94.88423280885489

In [36]:
rmse_1 = mean_squared_error(target_valid_1, predictions_valid_1)**0.5
print("RMSE модели 1:", rmse_1)

RMSE модели 1: 37.5794217150813


In [37]:
rmse_2 = mean_squared_error(target_valid_2, predictions_valid_2)**0.5
print("RMSE модели 2:", rmse_2)

RMSE модели 2: 0.8930992867756168


In [38]:
rmse_3 = mean_squared_error(target_valid_3, predictions_valid_3)**0.5
print("RMSE модели 3:", rmse_3)

RMSE модели 3: 40.02970873393434


In [39]:
table = [[predictions_valid_1.mean(), target_valid_1.mean(), rmse_1],
        [predictions_valid_2.mean(), target_valid_2.mean(), rmse_2],
        [predictions_valid_3.mean(), target_valid_3.mean(), rmse_3]]
columns = ['stock_predicted', 'stock', 'rmse']
index = ['region_1', 'region_2', 'region_3'] 
table_1 = pd.DataFrame(table, index, columns) 
display(table_1)

,stock_predicted,stock,rmse
region_1,92.592568,92.078597,37.579422
region_2,68.728547,68.723136,0.893099
region_3,94.965046,94.884233,40.029709


Среди двух регионов можно выделить 2 наиболее перспективных: 1 и 3.
___
Among the two regions, 2 most promising can be distinguished: 1 and 3.

## Подготовка к расчёту прибыли
___
## Preparation for profit calculation

In [40]:
BAR_1K = 450000
BUDGET = 10000000000
OILER_COUNT = 200
NO_LOSS_STOCK_VOL = BUDGET/OILER_COUNT/BAR_1K

In [41]:
print("Достаточный объём сырья для безубыточной разработки новой скважины:", NO_LOSS_STOCK_VOL)

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


In [42]:
table = [[df_1['product'].mean(), NO_LOSS_STOCK_VOL],
        [df_2['product'].mean(), NO_LOSS_STOCK_VOL],
        [df_3['product'].mean(), NO_LOSS_STOCK_VOL]]
columns = ['stock', 'no_loss_stock_vol']
index = ['region_1', 'region_2', 'region_3'] 
table_2 = pd.DataFrame(table, index, columns) 
display(table_2)

,stock,no_loss_stock_vol
region_1,92.500,111.111111
region_2,68.825,111.111111
region_3,95.000,111.111111


Средние данные опять лучше в 1 и 3 регионах.
___
The average data is again better in regions 1 and 3.

## Расчёт прибыли и рисков 
___
## Profit and Risk Calculation

In [43]:
def calc(target, predicted, count):
    predicted_200 = predicted.sort_values(ascending=False)
    target_200 = target[predicted_200.index][:count]
    income = target_200.sum() * BAR_1K - BUDGET
    return income

In [44]:
COUNT = 200

In [45]:
state = RandomState(12345) 

In [46]:
target_1_b = pd.Series(target_valid_1).reset_index(drop=True)
predict_1_b = pd.Series(predictions_valid_1).reset_index(drop=True)
values_1 = []
for i in range(1000):
    target_sub = target_1_b.sample(n=500, replace=True, random_state=state)
    predictions_sub = predict_1_b[target_sub.index]
    income_choice = calc(target_sub, predictions_sub, COUNT)
    values_1.append(income_choice)    
    value_1 = pd.Series(values_1)
    harm_risk_1 = (value_1[value_1 < 0].count()/value_1.count())*100
    mean_1 = value_1.mean()
    lower_1 = value_1.quantile(0.025)
    high_1 = value_1.quantile(0.975)
    

    

print('Средняя прибыль:', mean_1/1000000)
print('2,5% квантиль:', lower_1)
print('97,5% квантиль:', high_1)
print('Риск убытков:', harm_risk_1,"%")

Средняя прибыль: 425.93852691059203
2,5% квантиль: -102090094.83793654
97,5% квантиль: 947976353.3583689
Риск убытков: 6.0 %


In [47]:
target_2_b = pd.Series(target_valid_2).reset_index(drop=True)
predict_2_b = pd.Series(predictions_valid_2).reset_index(drop=True)
values_2 = []
for i in range(1000):
    target_sub = target_2_b.sample(n=500, replace=True, random_state=state)
    predictions_sub = predict_2_b[target_sub.index]
    income_choice = calc(target_sub, predictions_sub, COUNT)
    values_2.append(income_choice)    
    value_2 = pd.Series(values_2)
    harm_risk_2 = (value_2[value_2 < 0].count()/value_2.count())*100
    mean_2 = value_2.mean()
    lower_2 = value_2.quantile(0.025)
    high_2 = value_2.quantile(0.975)

print('Средняя прибыль:', mean_2/1000000)
print('2,5% квантиль:', lower_2)
print('97,5% квантиль:', high_2)
print('Риск убытков:', harm_risk_2,"%")

Средняя прибыль: 518.2594936973251
2,5% квантиль: 128123231.43308444
97,5% квантиль: 953612982.0669085
Риск убытков: 0.3 %


In [48]:
target_3_b = pd.Series(target_valid_3).reset_index(drop=True)
predict_3_b = pd.Series(predictions_valid_3).reset_index(drop=True)
values_3 = []
for i in range(1000):
    target_sub = target_3_b.sample(n=500, replace=True, random_state=state)
    predictions_sub = predict_3_b[target_sub.index]
    income_choice = calc(target_sub, predictions_sub, COUNT)
    values_3.append(income_choice)    
    value_3 = pd.Series(values_3)
    harm_risk_3 = (value_3[value_3 < 0].count()/value_3.count())*100
    mean_3 = value_3.mean()
    lower_3 = value_3.quantile(0.025)
    high_3 = value_3.quantile(0.975)

print('Средняя прибыль:', mean_3/1000000)
print('2,5% квантиль:', lower_3)
print('97,5% квантиль:', high_3)
print('Риск убытков:', harm_risk_3,"%")

Средняя прибыль: 420.19400534404986
2,5% квантиль: -115852609.16001143
97,5% квантиль: 989629939.8445739
Риск убытков: 6.2 %


In [49]:
table = [[mean_1, lower_1, high_1, harm_risk_1],
        [mean_2, lower_2, high_2, harm_risk_2],
        [mean_3, lower_3, high_3, harm_risk_3]]
columns = ['Средняя прибыль', '2,5% квантиль', '97,5% квантиль', 'Риск убытков']
index = ['region_1', 'region_2', 'region_3'] 
table_2 = pd.DataFrame(table, index, columns) 
display(table_2)

,Средняя прибыль,"2,5% квантиль","97,5% квантиль",Риск убытков
region_1,4.259385e+08,-1.020901e+08,9.479764e+08,6.0
region_2,5.182595e+08,1.281232e+08,9.536130e+08,0.3
region_3,4.201940e+08,-1.158526e+08,9.896299e+08,6.2


Вывод - наиболее перспективен регион под номером 2, так как его разработка обнаруживает минимальный риск (0.3%) и потенциальная средняя прибыль наиболее высока (518.2595). Два других региона обладают одинаково высоким уровнем риска убытков.
___
Conclusion - the region number 2 is the most promising, since its development reveals the minimum risk (0.3%) and the potential average profit is the highest (518.2595). The other two regions have an equally high risk of loss.